In [0]:
from pathlib import Path
import pandas as pd
from collections import defaultdict
import copy
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from datetime import datetime

In [0]:
# !tar xvf simple-examples.tgz

In [0]:
test_ptb = Path.cwd() / 'simple-examples/data/' / 'ptb.test.txt'
train_ptb = Path.cwd() / 'simple-examples/data/' / 'ptb.train.txt'
valid_ptb = Path.cwd() / 'simple-examples/data/' / 'ptb.valid.txt'

In [4]:
test_ptb.exists(), train_ptb.exists(), valid_ptb.exists() 

(True, True, True)

In [0]:
def read_ptb_file(ptb_file: Path):
    ptb_dict = defaultdict(int)
    one_line = []
    with ptb_file.open(mode='r') as f:
        for ind, sent in enumerate(f):
            sent_words = sent.strip().split()
            sent_words += ['<eos>']
            for word in sent_words:
                ptb_dict[word] += 1
            one_line += sent_words
    print(ind)
    return ptb_dict, one_line

In [88]:
test_dict, test_ol = read_ptb_file(test_ptb)
train_dict, train_ol = read_ptb_file(train_ptb)
valid_dict, valid_ol = read_ptb_file(valid_ptb)
len(test_ol), len(train_ol), len(valid_ol)

3760
42067
3369


(82430, 929589, 73760)

In [7]:
set(train_dict.keys()).issuperset(set(valid_dict.keys()))

True

In [0]:
word_to_ind = {k: ind for ind, (k, v) in enumerate(train_dict.items())}
ind_to_word = {v: k for k, v in word_to_ind.items()}

In [0]:
num_layers = 2 
emb_size = 200 
batch_size = 20 
num_steps = 35 
vocab_size = len(train_dict)
hidden_size = 200
max_grad_norm = 5

In [0]:
def indecise(one_line: list, word_to_ind: dict) -> list:
    return [word_to_ind[word] for word in one_line]

In [0]:
def batch_to_text(batch, ind_to_word):
    vect_func = np.vectorize(lambda x: ind_to_word[x])
    return vect_func(batch)

In [0]:
def generate_batches(ol, word_to_ind, batch_size=5, num_steps=3):
    ol = ol[0:(len(ol) // (batch_size * num_steps)) * batch_size * num_steps]
    ind_ol = np.array(indecise(ol, word_to_ind))
    ind_ol = ind_ol.reshape([batch_size, -1])
    num_batches = ind_ol.shape[1] // num_steps
    for i in range(num_batches):
        yield ind_ol[:, i * num_steps: (i+1) * num_steps]

In [0]:
gen_batches_X = generate_batches(ol=train_ol, word_to_ind=word_to_ind, batch_size=batch_size, num_steps=num_steps)
gen_batches_Y = generate_batches(ol=train_ol[1:], word_to_ind=word_to_ind, batch_size=batch_size, num_steps=num_steps)

In [0]:
x0 = next(gen_batches_X)
y0 = next(gen_batches_Y)

In [0]:
tf.reset_default_graph()

In [0]:
ph_input_seq = tf.placeholder(dtype=tf.int32, shape=(None, None), name='input_sequence') # bs, seq_len
ph_target_seq = tf.placeholder(dtype=tf.int32, shape=(None, None), name='target_sequence') # bs, seq_len
ph_lr = tf.placeholder(dtype=tf.float32, name='learning_rate')
ph_keep_prob = tf.placeholder(dtype=tf.float32, name='keep_probability')

In [17]:
word_embeddings = tf.get_variable(name='word_embs', 
                                  shape=[vocab_size, emb_size], 
                                  initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
batch_mat = tf.nn.embedding_lookup(word_embeddings, ph_input_seq, name='word_emb_lookup')  # bs, seq_len, emb_size 

In [0]:
# cell_state_1 = tf.placeholder(dtype=tf.float32, shape=(batch_size, hidden_size), name='cell_state_1')
# cell_state_2 = tf.placeholder(dtype=tf.float32, shape=(batch_size, hidden_size), name='cell_state_2')
# hidd_state_1 = tf.placeholder(dtype=tf.float32, shape=(batch_size, hidden_size), name='hidden_state_1')
# hidd_state_2 = tf.placeholder(dtype=tf.float32, shape=(batch_size, hidden_size), name='hidden_state_2')

# lstm_cell_tuple_c1 = tf.contrib.rnn.LSTMStateTuple(
#     cell_state_1,
#     hidd_state_1
# )

# lstm_cell_tuple_c2 = tf.contrib.rnn.LSTMStateTuple(
#     cell_state_2,
#     hidd_state_2
# )

In [0]:
# cell = tf.contrib.rnn.LSTMBlockCell(num_units=hidden_size)

In [0]:
# lstm_layers = [tf.contrib.rnn.LSTMBlockCell(num_units=size) for size in [hidden_size, hidden_size]]
# multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_layers)
# initial_state = multi_cell.zero_state(batch_size, dtype=tf.float32)
# outputs, state = tf.nn.dynamic_rnn(
#                               cell=multi_cell,
#                               inputs=batch_mat,
#                               dtype=tf.float32
#                               initial_state=initial_state)

In [0]:
# cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size)
# initial_state_cell = cell.zero_state(batch_size, dtype=tf.float32)
# outputs, state = tf.nn.dynamic_rnn(
#                               cell=cell,
#                               inputs=batch_mat,
#                               dtype=tf.float32,
#                               initial_state=initial_state_cell)

In [23]:
lstm_layers = [tf.nn.rnn_cell.LSTMCell(num_units=hidden_size) for _ in range(2)]
multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_layers)
initial_state_multi = multi_cell.zero_state(batch_size, dtype=tf.float32)
outputs, state = tf.nn.dynamic_rnn(
                              cell=multi_cell,
                              inputs=batch_mat,
                              dtype=tf.float32,
                              initial_state=initial_state_multi)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [0]:
# multi_cell.zero_state(batch_size=batch_size, dtype=tf.float32)

In [0]:
# [[batch_size, s] for s in multi_cell.state_size]

In [0]:
# outputs, state = tf.nn.dynamic_rnn(multi_cell, 
#                                    inputs=batch_mat, 
#                                    dtype=tf.float32, 
#                                    initial_state=(lstm_cell_tuple_c1, lstm_cell_tuple_c2))

In [0]:
logits = tf.contrib.layers.fully_connected(outputs,  # batch_size, num_steps, vocab_size
                                           num_outputs=vocab_size,
                                           activation_fn=None)

In [0]:
probabs = tf.nn.softmax(logits) # batch_size, num_steps, vocab_size

In [0]:
seq_mask = tf.sequence_mask(lengths=[num_steps] * batch_size, maxlen=num_steps, dtype=tf.float32)

In [0]:
losses = tf.contrib.seq2seq.sequence_loss(logits,
                                          ph_target_seq,
                                          weights=seq_mask,
                                          average_across_timesteps=True,
                                          average_across_batch=True)

In [31]:
tvars= tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(losses, tvars),
                                  max_grad_norm)
optimizer = tf.train.AdamOptimizer(learning_rate=ph_lr, )
train_op = optimizer.apply_gradients(zip(grads, tvars))

Instructions for updating:
Use tf.cast instead.


In [32]:

tvars

[<tf.Variable 'word_embs:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/lstm_cell/kernel:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/lstm_cell/bias:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'fully_connected/weights:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'fully_connected/biases:0' shape=(10000,) dtype=float32_ref>]

In [0]:
# traininig_op = optimizer.minimize(losses)

In [0]:
saver = tf.train.Saver()

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [0]:
summary_loss = tf.summary.scalar(name='scalar_loss', tensor=losses)

In [0]:
num_batches = len(train_ol) // (batch_size) // (num_steps)

In [0]:
def time_str():
    return datetime.now().replace(microsecond=0).isoformat().replace(':', '-')

In [46]:
n_epochs = 30
model_name = 'lm_ptb'
step = 0
lr = 1

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter(model_name + '/summaries%s' % time_str(), sess.graph)
    for i in range(n_epochs):
        gen_batches_X = generate_batches(ol=train_ol, word_to_ind=word_to_ind, batch_size=batch_size, num_steps=num_steps)
        gen_batches_Y = generate_batches(ol=train_ol[1:], word_to_ind=word_to_ind, batch_size=batch_size, num_steps=num_steps)
        with tqdm(total=num_batches) as p:
            last_state = None
            for x, y in zip(gen_batches_X, gen_batches_Y):
                if last_state is not None:
                  feed_dict = {initial_state_multi: last_state,
                                 ph_input_seq: x, 
                                 ph_target_seq: y,
                                 ph_lr: lr}
                else:
                  feed_dict = {
                                 ph_input_seq: x, 
                                 ph_target_seq: y,
                                 ph_lr: lr}
                    
                _, last_state = sess.run(
                    [train_op, state],
                    feed_dict=feed_dict)

                if step % 100 == 0:
                    perplex = losses.eval(
                        feed_dict=feed_dict
                    )
                    print(perplex)
                    summ = sess.run(summary_loss, 
                             feed_dict=feed_dict)
                    train_writer.add_summary(summ, global_step=step)
                p.update(1)
                step += 1
        if (i % 1 == 0):
            print('Epoch ', i)
            saver.save(sess,
                       model_name + '/model_nt/model',
                       global_step=i)

  0%|          | 1/1327 [00:00<15:05,  1.46it/s]

24.709757


  8%|▊         | 105/1327 [00:04<00:48, 25.43it/s]

107.2844


 15%|█▌        | 204/1327 [00:07<00:41, 27.10it/s]

...

27.453674


100%|█████████▉| 1323/1327 [00:46<00:00, 26.99it/s]

21.01209


100%|██████████| 1327/1327 [00:47<00:00, 28.18it/s]


Epoch  29


In [52]:
sess.close()
sess = tf.InteractiveSession()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
tf.reset_default_graph()

In [56]:
tf.trainable_variables()

[<tf.Variable 'word_embs:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/lstm_cell/kernel:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/lstm_cell/bias:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'fully_connected/weights:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'fully_connected/biases:0' shape=(10000,) dtype=float32_ref>]

In [55]:
new_saver = tf.train.import_meta_graph('./lm_ptb/model_nt/model-28.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('./lm_ptb/model_nt'))

INFO:tensorflow:Restoring parameters from ./lm_ptb/model_nt/model-29


In [0]:
gen_batches_X = generate_batches(ol=train_ol, word_to_ind=word_to_ind, batch_size=batch_size, num_steps=num_steps)
gen_batches_Y = generate_batches(ol=train_ol[1:], word_to_ind=word_to_ind, batch_size=batch_size, num_steps=num_steps)
x = next(gen_batches_X)
y = next(gen_batches_Y)
feed_dict = {
    ph_input_seq: x, 
    ph_target_seq: y,
    ph_lr: lr
}

In [0]:
prob = sess.run(probabs, feed_dict=feed_dict)

In [87]:
np.argwhere(prob[0, 1, :] > 0)

array([[  25],
       [  26],
       [  28],
       ...,
       [9954],
       [9956],
       [9968]])

In [79]:
0sess.run(tf.global_variables_initializer())

SyntaxError: ignored

In [64]:
word_embeddings.eval()

array([[-2.7464739e+01, -3.6920895e+01, -3.6771332e+01, ...,
         2.6273001e+01, -3.7348797e+01,  3.6746384e+01],
       [ 6.3345112e+01, -9.7019281e+00, -5.1374656e-01, ...,
        -1.0484942e+01,  1.3455794e+00,  6.3885632e+01],
       [ 3.1979662e+01, -3.2417019e+01,  2.2178928e+01, ...,
        -3.2884804e+01,  2.2129156e+01, -2.6629168e+01],
       ...,
       [-3.1723948e+01, -3.1731722e+01, -3.1821632e+01, ...,
        -3.1739698e+01, -3.1674166e+01, -3.1669527e+01],
       [ 3.0517769e+01,  3.0611116e+01,  3.0411850e+01, ...,
         3.0528505e+01, -3.0427708e+01, -3.0400440e+01],
       [ 7.7703960e-02, -5.8487877e-03,  3.9323784e-02, ...,
         5.2389242e-02, -7.4387550e-02,  1.6977310e-02]], dtype=float32)

In [0]:
stat = sess.run(state, feed_dict={ph_input_seq: x0, ph_target_seq: y0})

In [43]:
stat[1][1].shape

(20, 200)

In [44]:
losses[0].shape

ValueError: ignored

In [0]:
sess.run(multi_cell.initial_state)

In [0]:
losses[0].shape

In [0]:
np.array_equal(out[:, 2, :], stat[1])

In [0]:
stat[1].shape

In [0]:
stat[0].shape

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
n_inputs = 3
n_neurons = 5

In [0]:
X_0 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])
X_1 = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs])

In [0]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [0]:
output, states = tf.contrib.rnn.static_rnn(basic_cell, [X_0, X_1], dtype=tf.float32)

In [0]:
Y_0, Y_1 = output

In [0]:
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) 
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    out, stat = sess.run([output, states], feed_dict={X_0: X0_batch, X_1: X1_batch})

In [0]:
out[0].shape, np.array_equal(out[1], stat)

In [0]:
X0_batch.shape